# HyperDrive

### Create a workspace and an experiment

In [1]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
experiment_name = 'udacity-project'
exp=Experiment(ws, experiment_name)

run = exp.start_logging()

Workspace name: quick-starts-ws-132062
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-132062


### Create a Compute Cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "cpu-cluster-mla"

   # Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

    cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.


### Configure HyperDrive Run

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset
from azureml.data.datapath import DataPath

found = False
key = "iris-classification"
description_text = "classification of Iris dataset for capstone project"

if key in ws.datasets.keys(): 
        found = True        
        dataset = ws.datasets[key] 

if not found:
        # Create Hyperdrive Dataset and register it into Workspace
        url_path = 'https://raw.githubusercontent.com/dib1979/Temporary/main/IRIS.csv'
        dataset = TabularDatasetFactory.from_delimited_files(path=url_path)       
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                    name= key,
                    description= description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
#import numpy as np

from azureml.core import Environment
sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

from azureml.core import ScriptRunConfig
compute_target = ws.compute_targets['cpu-cluster-mla']
src = ScriptRunConfig(source_directory='.',
                      script='train1.py',
                      arguments=['--kernel', 'linear', '--penalty', 1.0],
                      compute_target=compute_target,
                      environment=sklearn_env)




In [5]:
run = exp.submit(src)
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: udacity-project_1609098382_ce3f169c
Web View: https://ml.azure.com/experiments/udacity-project/runs/udacity-project_1609098382_ce3f169c?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-132062/workspaces/quick-starts-ws-132062

Streaming azureml-logs/55_azureml-execution-tvmps_918ae27908ef1514a87ea0512e52590fdab157e68b9e3f5961f50a8c6684d8c2_d.txt

2020-12-27T19:46:46Z Starting output-watcher...
2020-12-27T19:46:46Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2020-12-27T19:46:46Z Executing 'Copy ACR Details file' on 10.0.0.4
2020-12-27T19:46:46Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_d97f122bdcaea8212b2eadb5cef56a6d
2c11b7cecaa5: Already exists
04637fa56252: Already exists
d6e6af23a0f3: Already exists
b4a424de92ad: Already exists
3e5d9ee64909: Already exists
3a846111ff22: Already exists
93a5020c6e19: Already exists
360

{'runId': 'udacity-project_1609098382_ce3f169c',
 'target': 'cpu-cluster-mla',
 'status': 'Completed',
 'startTimeUtc': '2020-12-27T19:46:42.203166Z',
 'endTimeUtc': '2020-12-27T19:50:15.674956Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '183835e8-607e-43ad-99c5-a373309d7b83',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train1.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--kernel', 'linear', '--penalty', '1'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpu-cluster-mla',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'environment': {'name': 'AzureML-Tutorial',
   'version': '60',
   'pyt

In [6]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice

param_sampling = RandomParameterSampling( {
    "--kernel": choice('linear', 'rbf', 'sigmoid'),
    "--penalty": choice(0.5, 1, 1.5)
    }
)

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

In [7]:
# start the HyperDrive run
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_4c4dcac6-312f-4f08-86a2-e0fce58ea3e2
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_4c4dcac6-312f-4f08-86a2-e0fce58ea3e2?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-132062/workspaces/quick-starts-ws-132062

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-27T19:51:06.763995][API][INFO]Experiment created<END>\n"<START>[2020-12-27T19:51:07.5205565Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-27T19:51:07.652453][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-12-27T19:51:07.976376][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_4c4dcac6-312f-4f08-86a2-e0fce58ea3e2
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_4c4dcac6-312f-4f08-86a2-e0fce58ea3e2?wsid=/subscriptions/f5091

### Get your best run and save the model from that run.

In [8]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_4c4dcac6-312f-4f08-86a2-e0fce58ea3e2_1

 Accuracy: 0.9777777777777777


In [9]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = best_run.register_model(model_name='iris-model', 
                           model_path='outputs/model.joblib',
                           model_framework=Model.Framework.SCIKITLEARN,
                           model_framework_version='0.19.1',
                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))
print(model.name, model.id, model.version, sep='\t')

iris-model	iris-model:1	1


In [10]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-132062', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-132062'), name=iris-model, id=iris-model:1, version=1, tags={}, properties={})


### Model Deployment

## This model isn't being deployed as its accuracy is less than the AutoML Model
### HYPERDRIVE MODEL - ACCURACY = 0.9777777777777777
### AUTOML MODEL [VOTINGENSEMBLE] - ACCURACY = 0.99047619